In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from loguru import logger
import pickle
import os 

X = pd.read_csv(r".\data\preprocessed_data.csv")
y_true = pd.read_csv(r".\data\y_true.csv")

In [2]:
fraud_percentage = y_true.value_counts(normalize=True)[1]
logger.info(f"Calculated fraud percentage in the dataset: {fraud_percentage:.4f} ({fraud_percentage*100:.2f}%)")

2025-06-19 23:00:53.986 | INFO     | __main__:<module>:2 - Calculated fraud percentage in the dataset: 0.0017 (0.17%)


In [3]:
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=fraud_percentage,  # Setting based on known fraud rate for initial evaluation
    random_state=42,
    n_jobs=-1
)

In [4]:
iso_forest.fit(X)

,n_estimators,100
,max_samples,'auto'
,contamination,np.float64(0....7485630620034)
,max_features,1.0
,bootstrap,False
,n_jobs,-1
,random_state,42
,verbose,0
,warm_start,False


In [5]:
predictions = iso_forest.predict(X)

In [7]:
anomaly_scores = iso_forest.decision_function(X)
logger.info(f"Shape of anomaly scores array: {anomaly_scores.shape}")

2025-06-19 23:06:47.540 | INFO     | __main__:<module>:2 - Shape of anomaly scores array: (284807,)


In [8]:
df = pd.read_csv(r".\data\creditcard.csv")
df['anomaly_prediction'] = predictions
df['anomaly_score'] = anomaly_scores

In [9]:
df['is_anomaly'] = df['anomaly_prediction'].apply(lambda x: 1 if x == -1 else 0)

In [10]:
df[['Amount', 'Class', 'anomaly_score', 'anomaly_prediction', 'is_anomaly']].head(10)

,Amount,Class,anomaly_score,anomaly_prediction,is_anomaly
0,149.62,0,0.283031,1,0
1,2.69,0,0.300354,1,0
2,378.66,0,0.211082,1,0
3,123.50,0,0.265810,1,0
4,69.99,0,0.283993,1,0
5,3.67,0,0.302089,1,0
6,4.99,0,0.290196,1,0
7,40.80,0,0.154330,1,0
8,93.20,0,0.274117,1,0
9,3.68,0,0.292775,1,0


In [13]:
predicted_anomalies = df[df['is_anomaly'] == 1]
print(f"\nNumber of transactions predicted as anomalies: {predicted_anomalies.shape[0]}")
print("First 10 transactions predicted as anomalies:")
print(predicted_anomalies[['Amount', 'Class', 'anomaly_score', 'is_anomaly']].head(10))


Number of transactions predicted as anomalies: 492
First 10 transactions predicted as anomalies:
       Amount  Class  anomaly_score  is_anomaly
1632  7712.43      0      -0.096466           1
2957     7.50      0      -0.002853           1
2963   544.62      0      -0.053904           1
5425   553.60      0      -0.031866           1
5534     5.49      0      -0.002688           1
5535     1.98      0      -0.006760           1
5827    28.62      0      -0.026201           1
6624    23.98      0      -0.008242           1
6812   845.73      0      -0.051911           1
7485  1895.88      0      -0.041394           1


In [19]:
# Finding Recall and Precision of the model.
# Finding how many of those 492 are actual frauds

print("\n--- Examining Overlap: Actual Frauds vs. Predicted Anomalies ---")

actual_frauds = df[df['Class'] == 1]
predicted_anomalies = df[df['is_anomaly'] == 1]

# True Positives (TP): Actual frauds that were correctly identified as anomalies
true_positives_df = df[(df['Class'] == 1) & (df['is_anomaly'] == 1)]
num_true_positives = true_positives_df.shape[0]

# False Positives (FP): Legitimate transactions incorrectly identified as anomalies
false_positives_df = df[(df['Class'] == 0) & (df['is_anomaly'] == 1)]
num_false_positives = false_positives_df.shape[0]

# False Negatives (FN): Actual frauds that were missed (identified as normal)
false_negatives_df = df[(df['Class'] == 1) & (df['is_anomaly'] == 0)]
num_false_negatives = false_negatives_df.shape[0]

# True Negatives (TN): Legitimate transactions correctly identified as normal
true_negatives_df = df[(df['Class'] == 0) & (df['is_anomaly'] == 0)]
num_true_negatives = true_negatives_df.shape[0]


--- Examining Overlap: Actual Frauds vs. Predicted Anomalies ---


In [20]:
print(f"Total Actual Fraudulent Transactions (Class=1): {actual_frauds.shape[0]}")
print(f"Total Transactions Predicted as Anomalies (is_anomaly=1): {predicted_anomalies.shape[0]}")
print("\n--- Breakdown of Predictions ---")
print(f"True Positives (Actual Fraud detected as Anomaly): {num_true_positives}")
print(f"False Positives (Legitimate detected as Anomaly): {num_false_positives}")
print(f"False Negatives (Actual Fraud missed): {num_false_negatives}")
print(f"True Negatives (Legitimate detected as Normal): {num_true_negatives}")

Total Actual Fraudulent Transactions (Class=1): 492
Total Transactions Predicted as Anomalies (is_anomaly=1): 492

--- Breakdown of Predictions ---
True Positives (Actual Fraud detected as Anomaly): 139
False Positives (Legitimate detected as Anomaly): 353
False Negatives (Actual Fraud missed): 353
True Negatives (Legitimate detected as Normal): 283962


In [23]:
print("\n--- Key Metrics for Anomaly Detection ---")

# Precision: Of all predicted anomalies, how many were truly fraudulent?
precision = num_true_positives / (num_true_positives + num_false_positives) if (num_true_positives + num_false_positives) > 0 else 0
print(f"Precision (Anomaly Class): {precision:.4f}")

# Recall: Of all actual fraudulent transactions, how many were detected?
recall = num_true_positives / (num_true_positives + num_false_negatives) if (num_true_positives + num_false_negatives) > 0 else 0
print(f"Recall (Anomaly Class): {recall:.4f}")

# F1-Score: Harmonic mean of Precision and Recall
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f"F1-Score (Anomaly Class): {f1:.4f}")



--- Key Metrics for Anomaly Detection ---
Precision (Anomaly Class): 0.2825
Recall (Anomaly Class): 0.2825
F1-Score (Anomaly Class): 0.2825


In [24]:
print("\n--- Sample of True Positives (Actual Frauds Detected) ---")
print(true_positives_df[['Amount', 'Class', 'anomaly_score', 'is_anomaly']].head())

print("\n--- Sample of False Positives (Legitimate Transactions Misclassified as Anomalies) ---")
print(false_positives_df[['Amount', 'Class', 'anomaly_score', 'is_anomaly']].head())



--- Sample of True Positives (Actual Frauds Detected) ---
      Amount  Class  anomaly_score  is_anomaly
8296     1.0      1      -0.032495           1
8335     1.0      1      -0.032264           1
8615     1.0      1      -0.044194           1
9035     1.0      1      -0.035869           1
9179     1.0      1      -0.040542           1

--- Sample of False Positives (Legitimate Transactions Misclassified as Anomalies) ---
       Amount  Class  anomaly_score  is_anomaly
1632  7712.43      0      -0.096466           1
2957     7.50      0      -0.002853           1
2963   544.62      0      -0.053904           1
5425   553.60      0      -0.031866           1
5534     5.49      0      -0.002688           1


In [25]:
print("\n--- Sample of False Negatives (Actual Frauds Missed) ---")
# These are the ones where Class=1 but is_anomaly=0
print(false_negatives_df[['Amount', 'Class', 'anomaly_score', 'is_anomaly']].head())


--- Sample of False Negatives (Actual Frauds Missed) ---
      Amount  Class  anomaly_score  is_anomaly
541     0.00      1       0.148459           0
623   529.00      1       0.194397           0
4920  239.93      1       0.185266           0
6108   59.00      1       0.080463           0
6329    1.00      1       0.062171           0


In [26]:
print("\n--- Finished with Detailed Anomaly Analysis ---")


--- Finished with Detailed Anomaly Analysis ---


### Interpretation
- Correctly identified the 492 most anomalous transactions based on the contamination parameter. 
    - This is why the precision and recall are identical: out of the 492 transactions flagged as anomalies, 28.25% of them were actual frauds.
    - And since we aimed for 492 anomalies, we only caught 28.25% of the total actual frauds.
- Low Precision / Recall
    - Precision: Out of all the transactions that Isolation Forest flagged as "anomalous", only about 28.25% were actually fraudulent. The remaining ~71.75% were legitimate transactions incorrectly flagged (False Positives). This highlights a significant "noise" or false alarm rate.
    - Recall: Only managed to capture 28.25% of the total actual fraudulent transactions present in the dataset. A large portion of real frauds were missed (False Negatives).

In [ ]:
df.to_csv(r".\data\predictions.csv", index=False)

In [ ]:
d = pd.read_csv(r".\data\predictions.csv")
d.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V24,V25,V26,V27,V28,Amount,Class,anomaly_prediction,anomaly_score,is_anomaly
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,1,0.283031,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,1,0.300354,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,1,0.211082,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,1,0.265810,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,1,0.283993,0
